In [ ]:
# Add analysis to the path
import sys
import pathlib
parent_dir = os.path.join(os.path.abspath(''), "..")
sys.path.append(parent_dir)

import parsers
from pprint import pprint
from aggregation import *
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font',**{'family': 'serif', 'size': 19})
rc('text', usetex=True)

path_to_results = os.path.normpath(os.path.join(parent_dir, "archive"))
print(path_to_results)
working_dir = os.path.normpath(os.path.join(parent_dir, "working"))
data = parsers.main(path_to_results, working_dir=working_dir)
print("Finished")

In [ ]:
def minimal(flt):
    return "{0:.1f}".format(flt).rstrip('0').rstrip('.')

def set_up_legend(cols, size=14):
    handles, labels = plt.gca().get_legend_handles_labels()
    handles = np.concatenate((handles[::2],handles[1::2]),axis=0)
    labels = np.concatenate((labels[::2],labels[1::2]),axis=0)
    plt.legend(handles, labels, loc="upper right", ncol=cols, fontsize=size)

def replica_groups_figure(test_id, replica_block_size=10, sampling_period=5.0, peak=None, num_cores=1):
    actual_replicas = len(data[test_id].replicas) // replica_block_size
    for i in range(actual_replicas):
        replica_nums = [i * replica_block_size + j for j in range(replica_block_size)]
        plot_cpu_for_hosts((data[test_id].replicas[i].single() for i in replica_nums), num_cores=num_cores, sampling_period=sampling_period)

        # Add peak if specified
        if peak is not None:
            plt.axhline(y=peak, color='gray', linestyle='dashed')

        plt.legend(loc="upper right")
        plt.gca().set_title(f"CPU \% for replicas {i * replica_block_size} - {(i + 1) * replica_block_size - 1} for test ({test_id})", pad=10)
        plt.xlabel(f"Time (seconds)")
        plt.ylabel(f"Avg CPU Utilization over {minimal(sampling_period)} sec. (\%)")
        plt.gca().set_ylim([-5,105])
        plt.gca().xaxis.labelpad = 5
        set_up_legend(cols=2, size=14)

def cpu_figure(hosts, title, show=True, sampling_period=5.0, num_cores=1, max_cpu=105):
    plot_cpu_for_hosts(hosts, sampling_period=sampling_period, num_cores=num_cores)

    plt.legend(loc="upper right")
    plt.gca().set_title(title, pad=10)
    plt.xlabel(f"Time (seconds)")
    plt.ylabel(f"Avg CPU Utilization over {minimal(sampling_period)} sec. (\%)")
    plt.gca().set_ylim([0, max_cpu])
    plt.gca().xaxis.labelpad = 5
    set_up_legend(cols=2, size=14)

    if show:
        plt.show()

def plot_cpu_for_hosts(hosts, sampling_period=5.0, num_cores=1):
    for i, host in enumerate(hosts):
        df = aggregate_cpu(host, sampling_period=sampling_period)
        plt.plot([t / 1E3 for t in normalize(df["time"])], [c / num_cores for c in df["cpu"]], label=str(i))


In [ ]:
def make_single_cpu_subplot(test, peak):
    replica_groups_figure(test, replica_block_size = 20, sampling_period=5.0, peak=peak, num_cores=4)

fig = plt.figure(figsize=(17.5, 5))
plt.subplot(1, 2, 1)
make_single_cpu_subplot("d-rc-50", peak=50)
plt.subplot(1, 2, 2)
make_single_cpu_subplot("d-rc-100", peak=100)
plt.savefig('evaluation_50_100.pdf', bbox_inches='tight')

In [ ]:
def make_host_cpu_subplot(test, host, replicas):
    replica_nums = range(replicas)
    hosts = [data[test].replicas[i].hosts[host] for i in range(replicas)]
    title=f"CPU \% for auth in replicas {0} - {replicas - 1} for test ({test})"
    cpu_figure(hosts, title=title, show=False, sampling_period=10.0, max_cpu=40)

fig = plt.figure(figsize=(17.5, 5))
plt.subplot(1, 2, 1)
make_host_cpu_subplot("ii-rc-s", "auth", 4)
plt.subplot(1, 2, 2)
make_host_cpu_subplot("ii-rc-b", "auth", 4)
plt.savefig('evaluation_s_b.pdf', bbox_inches='tight')